<a href="https://colab.research.google.com/github/Juniorbasck/Machine-Learning/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Importamos a biblioteca transformers para acessar o modelo e o tokenizador DialoGPT, assim como torch para manipulação dos tensores.



In [2]:
import pandas as pd

dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}

df_status_pedidos = pd.DataFrame(dados_pedidos)
df_status_pedidos


,numero_pedido,status
0,12345,Shipped
1,67890,Processing
2,11121,Delivered
3,22232,Cancelled


Criamos um DataFrame com os números e status dos pedidos para simular uma base de dados.

In [3]:
def verificar_status_pedido(numero_pedido):
    try:
        status = df_status_pedidos[df_status_pedidos["numero_pedido"] == numero_pedido]["status"].iloc[0]
        return f"The status of your order {numero_pedido} is: {status}"
    except:
        return "Order number not found. Please check and try again."


Descrição: Definimos uma função que busca o status do pedido no DataFrame, retornando uma mensagem caso o pedido não seja encontrado.



In [4]:
palavras_chave_status = ["order", "order status", "status of my order", "check my order", "track my order", "order update"]


Especificamos palavras-chave que, quando detectadas, ativarão a consulta do status do pedido.



In [ ]:
ids_historico_chat = None

while True:
    input_usuario = input("You: ")

    if input_usuario.lower() in ['exit', 'quit', 'stop']:
        print("Bot: Goodbye!")
        break

    if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
        numero_pedido = input('Could you please enter your order number?')
        resposta = verificar_status_pedido(numero_pedido)
    else:
        novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

        if ids_historico_chat is not None:
            bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)
        else:
            bot_input_ids = novo_usuario_input_ids

        ids_historico_chat = model.generate(
            bot_input_ids,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id
        )

        resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    print(f'Bot: {resposta}')


Implementamos o loop de interação com o usuário. Quando o usuário menciona uma palavra-chave de consulta de pedidos, o chatbot solicita o número do pedido e exibe o status. Em outros casos, o modelo responde com base no contexto de conversa.

